# FASA Enhanced Strategy v2 - Dynasty Analytics

Leverages all 79 columns from enhanced `mrt_fasa_targets`:
- Dynasty aging curves & sustainability
- Market efficiency vs KTC
- League VoR context
- Multiple bid strategies
- **Balanced across QB/RB/WR/TE** (not RB-focused)

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Load FASA targets
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
fasa_path = repo_root / 'data' / 'raw' / 'marts' / 'mrt_fasa_targets' / 'dt=2025-10-30' / 'data.parquet'
fasa_targets = pd.read_parquet(fasa_path)

print(f"✅ {len(fasa_targets)} targets with {len(fasa_targets.columns)} enhanced columns")
print(f"Positions: {dict(fasa_targets['position'].value_counts())}")
print(f"Week {fasa_targets['current_week'].iloc[0]} | {fasa_targets['asof_date'].iloc[0]}")

## Top 20 Targets (All Positions)
Ranked by **enhanced_value_score_v2** (not RB-biased)

In [ ]:
top20 = fasa_targets.nlargest(20, 'enhanced_value_score_v2')[[
    'priority_rank_overall', 'player_name', 'position', 'nfl_team',
    'projected_ppg_ros', 'enhanced_value_score_v2', 'pts_above_league_replacement',
    'sustainability_score', 'bid_confidence_v3', 'suggested_bid_1yr', 'suggested_bid_dynasty_3yr'
]].round(1)

display(top20)

## Position Breakdowns

In [ ]:
cols = ['priority_rank_at_position', 'player_name', 'nfl_team', 'age',
        'projected_ppg_ros', 'weeks_remaining', 'pts_above_league_replacement',
        'sustainability_score', 'bid_confidence_v3', 'suggested_bid_1yr', 'suggested_bid_dynasty_3yr']

for pos, n in [('RB', 10), ('WR', 10), ('TE', 8), ('QB', 5)]:
    print(f"\n{'='*120}\n{pos} TOP {n}\n{'='*120}")
    display(fasa_targets[fasa_targets['position']==pos].head(n)[cols].round(1))

## Dynasty Analysis

In [ ]:
# High sustainability targets
dynasty = fasa_targets[
    (fasa_targets['sustainability_score'] >= 0.6) & (fasa_targets['dynasty_3yr_value'].notna())
].nlargest(12, 'dynasty_3yr_value')[[
    'player_name', 'position', 'age', 'sustainability_score', 'dynasty_3yr_value',
    'suggested_bid_dynasty_3yr', 'suggested_bid_1yr', 'age_decline_risk_score'
]].round(1)

print("\n🌟 BEST DYNASTY VALUES (3yr projection)")
display(dynasty)

## League VoR Context

In [ ]:
vor = fasa_targets[fasa_targets['pts_above_league_replacement'].notna()].nlargest(12, 'pts_above_league_replacement')[[
    'player_name', 'position', 'projected_ppg_ros', 'league_replacement_level_ppg',
    'pts_above_league_replacement', 'hypothetical_league_rank', 'hypothetical_percentile'
]].round(1)

print("\n📊 BEST LEAGUE VoR (above YOUR league replacement)")
display(vor)

## Bid Strategy Matrix

In [ ]:
bids = fasa_targets.nlargest(12, 'enhanced_value_score_v2')[[
    'player_name', 'position', 'projected_ppg_ros', 'bid_confidence_v3',
    'suggested_bid_1yr', 'suggested_bid_dynasty_3yr', 'suggested_bid_market_adjusted',
    'suggested_bid_contender', 'suggested_bid_rebuilder'
]].round(0)

print("\n🎯 BID STRATEGIES (Choose based on team context)")
display(bids)
print("\n1yr=win-now | dynasty_3yr=long-term | contender=aggressive | rebuilder=conservative")

## Final Recommendations

In [ ]:
# Change team_context to: 'contender', 'balanced', or 'rebuilder'
team_context = 'contender'

bid_map = {
    'contender': 'suggested_bid_contender',
    'balanced': 'suggested_bid_dynasty_3yr',
    'rebuilder': 'suggested_bid_rebuilder'
}

final = fasa_targets[fasa_targets[bid_map[team_context]].notna()].nlargest(15, 'enhanced_value_score_v2')[[
    'priority_rank_overall', 'player_name', 'position', 'nfl_team',
    'projected_ppg_ros', 'enhanced_value_score_v2', 'bid_confidence_v3', bid_map[team_context]
]].rename(columns={bid_map[team_context]: 'bid'}).round(0)

print(f"\n📋 SEALED BID SHEET ({team_context.upper()})\n{'='*100}")
display(final)

print("\n🎯 TOP 3:")
for i in range(min(3, len(final))):
    r = final.iloc[i]
    print(f"  {i+1}. {r['player_name']} ({r['position']}) - ${r['bid']:.0f} - {r['bid_confidence_v3']}")